In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from datetime import datetime
from datetime import timedelta
import requests
import pprint
import re
import seaborn as sns
sns.set_style('darkgrid')

%matplotlib inline

import math
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm

In [2]:
time1 = datetime.now()
df = pd.read_csv('flight_data.csv')
# p = 1  # p% of lines
# filename = 'flight_data.csv'
# random.seed(234)

# # if random from [0,1] > p, skip row
# df = pd.read_csv(
#          filename,
#          header=0, 
#          skiprows=lambda i: i>0 and random.random() > p/100
# )
time2 = datetime.now()
print('Time cost: ', time2-time1)

C:\Users\abrea\.julia\packages\Conda\hsaaN\deps\usr\lib\site-packages\IPython\core\interactiveshell.py:3018: DtypeWarning: Columns (48,77,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Time cost:  0:02:46.306142


In [3]:
df = df[(df['ORIGIN'] != 'TKI') & (df['TAIL_NUM'] != 'PLANET') & (df['TAIL_NUM'] != 'N101NK') & (df['TAIL_NUM'] != 'N999FR') & (df['TAIL_NUM'] != 'N700TA') & (df['TAIL_NUM'] != 'N187SW') & (df['TAIL_NUM'] != 'N399FR')]

In [ ]:
wban_iata = pd.read_csv('../Weather Data/wban_to_iata.csv')

weather = pd.read_csv('../Weather Data/data/weather_pre.csv')

In [ ]:
weather['WBAN'] = weather['STATION'].apply(lambda x: int(x[5:]))

In [ ]:
# For delayed flights
delayed_flight = df[df['DEP_DELAY'] > 15].reset_index(drop=True)
df_flight_time = pd.DataFrame()
df_flight_time['date'] = delayed_flight['FL_DATE']
df_flight_time['time'] = delayed_flight['CRS_DEP_TIME']
df_flight_time['origin'] = delayed_flight['ORIGIN']
df_flight_time['delay'] = delayed_flight['DEP_DELAY']
df_flight_time['time'] = df_flight_time['time'].apply(lambda x: str(x)[:-2]+':'+str(x)[-2:]+':00' if x >= 100 else '00:'+str(x)+':00')
df_flight_time['datetime'] = df_flight_time['date'].map(str) + '/' + df_flight_time['time']
df_flight_time['datetime'] = pd.to_datetime(df_flight_time['datetime'], format='%Y-%m-%d/%H:%M:%S')
df_flight_time = df_flight_time.merge(wban_iata, left_on='origin', right_on='IATA', how='inner').drop(columns=['origin'])

In [ ]:
cancalled_flight = df[(df['CANCELLED'] == 1) & (df['YEAR'] == 2017) & (df['MONTH'] == 10) & (df['DAY_OF_MONTH'] >= 9)& (df['DAY_OF_MONTH'] <= 12)].reset_index(drop=True)

In [ ]:
# For cancelled flights
# cancalled_flight = df[df['CANCELLED'] == 1].reset_index(drop=True)
df_flight_time = pd.DataFrame()
df_flight_time['date'] = cancalled_flight['FL_DATE']
df_flight_time['time'] = cancalled_flight['CRS_DEP_TIME']
df_flight_time['origin'] = cancalled_flight['ORIGIN']
df_flight_time['time'] = df_flight_time['time'].apply(lambda x: str(x)[:-2]+':'+str(x)[-2:]+':00' if x >= 100 else '00:'+str(x)+':00')
df_flight_time['datetime'] = df_flight_time['date'].map(str) + '/' + df_flight_time['time']
df_flight_time['datetime'] = pd.to_datetime(df_flight_time['datetime'], format='%Y-%m-%d/%H:%M:%S')
df_flight_time = df_flight_time.merge(wban_iata, left_on='origin', right_on='IATA', how='inner').drop(columns=['origin'])

In [ ]:
weather = weather[(weather['REPORTTPYE'] == 'FM-15') | (weather['REPORTTPYE'] == 'FM-16')]
weather['DATE'] = pd.to_datetime(weather['DATE'], format='%Y-%m-%d %H:%M')

In [ ]:
def find_closest_date(timepoint, time_series):
    # takes a pd.Timestamp() instance and a pd.Series with dates in it
    # calcs the delta between `timepoint` and each date in `time_series`
    # returns the closest date and optionally the number of days in its time delta
    timepoint = np.datetime64(timepoint)
    deltas = np.abs(np.subtract(time_series.values, timepoint))
    idx_closest_date = np.argmin(deltas)
    return time_series.index[idx_closest_date]

In [ ]:
# For delayed flights
time1 = datetime.now()
res_dict = dict()
minutes_dict = dict()
pattern = re.compile(r'\d{2}')
for i in range(df_flight_time.shape[0]):
    match = []
#     if df_flight_time.loc[i, 'WBAN'] == 94850:
#         continue
    flight_weather = weather.loc[find_closest_date(df_flight_time.loc[i, 'datetime'], weather[weather['WBAN'] == df_flight_time.loc[i, 'WBAN']].DATE)]
    if isinstance(flight_weather['HOURLYPRSENTWEATHERTYPE'], str):
        if re.search(r'\|.*?(([A-Za-z]{2,}:(\d{2})\s)*)\|', flight_weather['HOURLYPRSENTWEATHERTYPE']):
            string = re.search(r'\|.*?(([A-Za-z]{2,}:(\d{2})\s)*)\|', flight_weather['HOURLYPRSENTWEATHERTYPE']).group(0)
            match = pattern.findall(string)
    if len(match) == 0:
        if 0 in list(res_dict.keys()):
            res_dict[0] += 1
            minutes_dict[0] += df_flight_time.loc[i, 'delay']
        else:
            res_dict[0] = 1
            minutes_dict[0] = df_flight_time.loc[i, 'delay']
    else:
        for num in match:
            if int(num) in list(res_dict.keys()):
                res_dict[int(num)] += 1
                minutes_dict[int(num)] += df_flight_time.loc[i, 'delay']
            else:
                res_dict[int(num)] = 1
                minutes_dict[int(num)] = df_flight_time.loc[i, 'delay']
time2 = datetime.now()
print('Time cost: ', time2-time1)

In [ ]:
# For cancelled flights
time1 = datetime.now()
res_dict = dict()
# minutes_dict = dict()
pattern = re.compile(r'\d{2}')
for i in range(df_flight_time.shape[0]):
    match = []
#     if df_flight_time.loc[i, 'WBAN'] == 94850:
#         continue
    flight_weather = weather.loc[find_closest_date(df_flight_time.loc[i, 'datetime'], weather[weather['WBAN'] == df_flight_time.loc[i, 'WBAN']].DATE)]
    if isinstance(flight_weather['HOURLYPRSENTWEATHERTYPE'], str):
        if re.search(r'\|.*?(([A-Za-z]{2,}:(\d{2})\s)*)\|', flight_weather['HOURLYPRSENTWEATHERTYPE']):
            string = re.search(r'\|.*?(([A-Za-z]{2,}:(\d{2})\s)*)\|', flight_weather['HOURLYPRSENTWEATHERTYPE']).group(0)
            match = pattern.findall(string)
    if len(match) == 0:
        if 0 in list(res_dict.keys()):
            res_dict[0] += 1
#             minutes_dict[0] += df_flight_time.loc[i, 'delay']
        else:
            res_dict[0] = 1
#             minutes_dict[0] = df_flight_time.loc[i, 'delay']
    else:
        for num in match:
            if int(num) in list(res_dict.keys()):
                res_dict[int(num)] += 1
#                 minutes_dict[int(num)] += df_flight_time.loc[i, 'delay']
            else:
                res_dict[int(num)] = 1
#                 minutes_dict[int(num)] = df_flight_time.loc[i, 'delay']
time2 = datetime.now()
print('Time cost: ', time2-time1)

In [ ]:
avr_dict = dict()
for i in list(res_dict.keys()):
    avr_dict[i] = minutes_dict[i] / res_dict[i]

In [ ]:
weather_type_df = pd.read_csv('../Weather Data/weather_type_codes.csv', sep='\t')

In [ ]:
new_avr_dict = dict()
for i in range(weather_type_df.shape[0]):
    if weather_type_df.loc[i, 'weather code'] in list(avr_dict.keys()):
        new_avr_dict[weather_type_df.loc[i, 'weather type']] = avr_dict[weather_type_df.loc[i, 'weather code']]
new_avr_dict

In [ ]:
new_res_dict = dict()
for i in range(weather_type_df.shape[0]):
    if weather_type_df.loc[i, 'weather code'] in list(res_dict.keys()):
        new_res_dict[weather_type_df.loc[i, 'weather type']] = res_dict[weather_type_df.loc[i, 'weather code']]
new_res_df = pd.DataFrame.from_dict(new_res_dict, orient='index', columns=['count']).sort_values(by='count')
new_res_df

In [ ]:
new_avr_df = pd.DataFrame.from_dict(new_avr_dict, orient='index', columns=['average delay minutes']).sort_values(by='average delay minutes')
new_avr_df.plot(kind='barh', title='Average Delay Minutes of Different Weather Types', figsize=(10,7), fontsize=14)

In [ ]:
new_res_df = pd.DataFrame.from_dict(new_res_dict, orient='index', columns=['count']).sort_values(by='count')
new_res_df.plot(kind='barh', title='Count of Cancelled Flights of Different Weather Types', figsize=(10,7), fontsize=14)

In [ ]:
# generate delay_and_weather_type.csv
weather_delay_df = pd.concat([new_res_df, new_avr_df], axis=1).sort_values(by='average delay minutes', ascending=False).reset_index()
weather_delay_df = weather_delay_df.rename(columns={'index': 'weather_type'})
weather_delay_df.to_csv('../Weather Data/data/delay_and_weather_type.csv', index=False)

# Delay / carrier and/or aircraft correlation


In [ ]:
tail_num_df = pd.read_csv('./tail_num/cleaned_tail_num_with_model_2.csv')

In [ ]:
df = df.merge(tail_num_df.iloc[:, :-1], left_on='TAIL_NUM', right_on='tail_num', how='inner')

In [ ]:
# For delayed flights
delayed_flight = df[df['DEP_DELAY'] > 15].reset_index(drop=True)
# delayed_flight = df[(df['CARRIER_DELAY'] > 0) & (df['DEP_DELAY'] > 15)].reset_index(drop=True)
df_flight_time = pd.DataFrame()
df_flight_time['date'] = delayed_flight['FL_DATE']
df_flight_time['time'] = delayed_flight['CRS_DEP_TIME']
df_flight_time['tail_num'] = delayed_flight['TAIL_NUM']
df_flight_time['delay'] = delayed_flight['DEP_DELAY']
df_flight_time['carrier'] = delayed_flight['OP_CARRIER']
df_flight_time['time'] = df_flight_time['time'].apply(lambda x: str(x)[:-2]+':'+str(x)[-2:]+':00' if x >= 100 else '00:'+str(x)+':00')
df_flight_time['datetime'] = df_flight_time['date'].map(str) + '/' + df_flight_time['time']
df_flight_time['datetime'] = pd.to_datetime(df_flight_time['datetime'], format='%Y-%m-%d/%H:%M:%S')
df_flight_time = df_flight_time.merge(tail_num_df.iloc[:, :-1], on='tail_num', how='inner')
df_flight_time['age'] = df_flight_time['datetime'].dt.year - df_flight_time['year']

In [ ]:
df['age'] = df['YEAR'] - df['year']

In [ ]:
delayed_flight.loc[5, 'DEP_DELAY']

In [ ]:
delayed_flight.head(10).iloc[:, 25:45]

In [ ]:
delayed_flight.head(10).iloc[:, 56:61]

In [ ]:
delayed_flight.iloc[14164, 25:45]

In [ ]:
delayed_flight[delayed_flight['SECURITY_DELAY'] > 0].iloc[:, 56:61]

In [ ]:
delayed_flight.columns.values

## Delay to plane age

In [ ]:
all_flights_age_count = np.array(df.groupby('age').DEP_DELAY.count().tolist()[1:])
delay_flights_age_count = np.array(df_flight_time.groupby('age').delay.count().tolist()[1:])
plt.plot(range(0,33), delay_flights_age_count / all_flights_age_count)
plt.xlabel('Age of Aircraft', fontsize=20)
plt.ylabel('Delay Percentage', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
delayed_flight_age = delayed_flight.groupby('age').DEP_DELAY.mean()
plt.plot(delayed_flight_age.index, delayed_flight_age.values)
plt.xlabel('Age of Aircraft', fontsize=20)
plt.ylabel('Average Delay Minutes', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

## Delay to carrier

In [ ]:
all_flights_carrier_count = np.array(df.groupby('OP_CARRIER').DEP_DELAY.count().tolist())
delay_flights_carrier_count = np.array(df_flight_time.groupby('carrier').delay.count().tolist())
carrier_names = df.groupby('OP_CARRIER').DEP_DELAY.count().index.tolist()
carrier_delay_percentage = pd.DataFrame(delay_flights_carrier_count / all_flights_carrier_count, index=carrier_names, columns=['delay_percentage'])
carrier_delay_percentage = carrier_delay_percentage.sort_values(by='delay_percentage')
plt.bar(carrier_delay_percentage.index, carrier_delay_percentage.delay_percentage)
plt.xticks(delayed_flight_carrier.index, fontsize=15)
plt.xlabel('Carrier', fontsize=20)
plt.ylabel('Delay Percentage', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
delayed_flight_carrier = delayed_flight.groupby('OP_CARRIER').DEP_DELAY.mean().sort_values()
plt.bar(delayed_flight_carrier.index, delayed_flight_carrier.values)
plt.xticks(delayed_flight_carrier.index, fontsize=15)
plt.xlabel('Carrier', fontsize=20)
plt.ylabel('Average Delay Minutes', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

## Delay to plane manufacturer

In [ ]:
all_flights_mfr_count = np.array(df.groupby('mfr_name').DEP_DELAY.count().tolist())
delay_flights_mfr_count = np.array(df_flight_time.groupby('mfr_name').delay.count().tolist())
mfr_names = df.groupby('mfr_name').DEP_DELAY.count().index.tolist()
mfr_delay_percentage = pd.DataFrame(delay_flights_mfr_count / all_flights_mfr_count, index=mfr_names, columns=['delay_percentage'])
mfr_delay_percentage = mfr_delay_percentage.sort_values(by='delay_percentage')
plt.bar(mfr_delay_percentage.index, mfr_delay_percentage.delay_percentage)
plt.xticks(mfr_delay_percentage.index)
plt.xlabel('Manufacturer', fontsize=20)
plt.ylabel('Delay Percentage', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
delayed_flight_mfr = delayed_flight.groupby('mfr_name').DEP_DELAY.mean().sort_values()
plt.bar(delayed_flight_mfr.index, delayed_flight_mfr.values)
plt.xticks(delayed_flight_mfr.index)
plt.xlabel('Manufacturer', fontsize=20)
plt.ylabel('Average Delay Minutes', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

## Delay to aircraft model

In [ ]:
all_flights_model_count = np.array(df.groupby('model_2').DEP_DELAY.count().tolist())
delay_flights_model_count = np.array(df_flight_time.groupby('model_2').delay.count().tolist())
model_names = df.groupby('model_2').DEP_DELAY.count().index.tolist()
model_delay_percentage = pd.DataFrame(delay_flights_model_count / all_flights_model_count, index=model_names, columns=['delay_percentage'])
model_delay_percentage = model_delay_percentage.sort_values(by='delay_percentage')
plt.bar(model_delay_percentage.index, model_delay_percentage.delay_percentage)
plt.xticks(model_delay_percentage.index, rotation=90, fontsize=15)
plt.xlabel('Aircraft Model', fontsize=20)
plt.ylabel('Delay Percentage', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
delayed_flight_model = delayed_flight.groupby('model_2').DEP_DELAY.mean().sort_values()
plt.bar(delayed_flight_model.index, delayed_flight_model.values)
plt.xticks(delayed_flight_model.index, rotation=90)
plt.xlabel('Aircraft Model', fontsize=20)
plt.ylabel('Average Delay Minutes', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

## Delay to engine manufacturer

In [ ]:
all_flights_eng_mfr_count = np.array(df.groupby('eng_mfr_name').DEP_DELAY.count().tolist())
delay_flights_eng_mfr_count = np.array(df_flight_time.groupby('eng_mfr_name').delay.count().tolist())
eng_mfr_names = df.groupby('eng_mfr_name').DEP_DELAY.count().index.tolist()
eng_mfr_delay_percentage = pd.DataFrame(delay_flights_eng_mfr_count / all_flights_eng_mfr_count, index=eng_mfr_names, columns=['delay_percentage'])
eng_mfr_delay_percentage = eng_mfr_delay_percentage.sort_values(by='delay_percentage')
plt.bar(eng_mfr_delay_percentage.index, eng_mfr_delay_percentage.delay_percentage)
plt.xticks(eng_mfr_delay_percentage.index, rotation=45)
plt.xlabel('Engine Manufacturer')
plt.ylabel('Delay Percentage')

In [ ]:
delayed_flight_eng_mfr = delayed_flight.groupby('eng_mfr_name').DEP_DELAY.mean().sort_values()
plt.bar(delayed_flight_eng_mfr.index, delayed_flight_eng_mfr.values)
plt.xticks(delayed_flight_eng_mfr.index, rotation=45)
plt.xlabel('Engine Manufacturer')
plt.ylabel('Average Delay Minutes')

## Delay to engine model

In [ ]:
all_flights_eng_model_count = np.array(df.groupby('eng_model').DEP_DELAY.count().tolist())
delay_flights_eng_model_count = np.array(df_flight_time.groupby('eng_model').delay.count().tolist())
eng_model_names = df.groupby('eng_model').DEP_DELAY.count().index.tolist()
eng_model_delay_percentage = pd.DataFrame(delay_flights_eng_model_count / all_flights_eng_model_count, index=eng_model_names, columns=['delay_percentage'])
eng_model_delay_percentage = eng_model_delay_percentage.sort_values(by='delay_percentage')
plt.bar(eng_model_delay_percentage.index, eng_model_delay_percentage.delay_percentage)
plt.xticks(eng_model_delay_percentage.index, rotation=90, fontsize=15)
plt.xlabel('Engine Model', fontsize=20)
plt.ylabel('Delay Percentage', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
delayed_flight_eng_model = delayed_flight.groupby('eng_model').DEP_DELAY.mean().sort_values()
plt.bar(delayed_flight_eng_model.index, delayed_flight_eng_model.values)
plt.xticks(delayed_flight_eng_model.index, rotation=90, fontsize=15)
plt.xlabel('Engine Model', fontsize=20)
plt.ylabel('Average Delay Minutes', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

# Carrier Delay

In [ ]:
# For delayed flights
# delayed_flight = df[df['DEP_DELAY'] > 15].reset_index(drop=True)
delayed_flight = df[(df['CARRIER_DELAY'] > 0) & (df['DEP_DELAY'] > 15)].reset_index(drop=True)
df_flight_time = pd.DataFrame()
df_flight_time['date'] = delayed_flight['FL_DATE']
df_flight_time['time'] = delayed_flight['CRS_DEP_TIME']
df_flight_time['tail_num'] = delayed_flight['TAIL_NUM']
df_flight_time['delay'] = delayed_flight['CARRIER_DELAY']
df_flight_time['carrier'] = delayed_flight['OP_CARRIER']
df_flight_time['time'] = df_flight_time['time'].apply(lambda x: str(x)[:-2]+':'+str(x)[-2:]+':00' if x >= 100 else '00:'+str(x)+':00')
df_flight_time['datetime'] = df_flight_time['date'].map(str) + '/' + df_flight_time['time']
df_flight_time['datetime'] = pd.to_datetime(df_flight_time['datetime'], format='%Y-%m-%d/%H:%M:%S')
df_flight_time = df_flight_time.merge(tail_num_df.iloc[:, :-1], on='tail_num', how='inner')
df_flight_time['age'] = df_flight_time['datetime'].dt.year - df_flight_time['year']

## Carrier Delay to Age

In [ ]:
all_flights_age_count = np.array(df.groupby('age').DEP_DELAY.count().tolist()[1:])
delay_flights_age_count = np.array(df_flight_time.groupby('age').delay.count().tolist()[1:])
plt.plot(range(0,33), delay_flights_age_count / all_flights_age_count)
plt.xlabel('Age of Aircraft', fontsize=20)
plt.ylabel('Carrier Delay Percentage', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
delayed_flight_age = delayed_flight.groupby('age').CARRIER_DELAY.mean()
plt.plot(delayed_flight_age.index, delayed_flight_age.values)
plt.xlabel('Age of Aircraft', fontsize=20)
plt.ylabel('Average Carrier Delay Minutes', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

## Carrier Delay to carrier

In [ ]:
all_flights_carrier_count = np.array(df.groupby('OP_CARRIER').DEP_DELAY.count().tolist())
delay_flights_carrier_count = np.array(df_flight_time.groupby('carrier').delay.count().tolist())
carrier_names = df.groupby('OP_CARRIER').CARRIER_DELAY.count().index.tolist()
carrier_delay_percentage = pd.DataFrame(delay_flights_carrier_count / all_flights_carrier_count, index=carrier_names, columns=['delay_percentage'])
carrier_delay_percentage = carrier_delay_percentage.sort_values(by='delay_percentage')
plt.bar(carrier_delay_percentage.index, carrier_delay_percentage.delay_percentage)
plt.xticks(delayed_flight_carrier.index, fontsize=15)
plt.xlabel('Carrier', fontsize=20)
plt.ylabel('Carrier Delay Percentage', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
delayed_flight_carrier = delayed_flight.groupby('OP_CARRIER').CARRIER_DELAY.mean().sort_values()
plt.bar(delayed_flight_carrier.index, delayed_flight_carrier.values)
plt.xticks(delayed_flight_carrier.index, fontsize=15)
plt.xlabel('Carrier', fontsize=20)
plt.ylabel('Average Carrier Delay Minutes', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

## Carrier Delay to plane manufacturer

In [ ]:
all_flights_mfr_count = np.array(df.groupby('mfr_name').DEP_DELAY.count().tolist())
delay_flights_mfr_count = np.array(df_flight_time.groupby('mfr_name').delay.count().tolist())
mfr_names = df.groupby('mfr_name').DEP_DELAY.count().index.tolist()
mfr_delay_percentage = pd.DataFrame(delay_flights_mfr_count / all_flights_mfr_count, index=mfr_names, columns=['delay_percentage'])
mfr_delay_percentage = mfr_delay_percentage.sort_values(by='delay_percentage')
plt.bar(mfr_delay_percentage.index, mfr_delay_percentage.delay_percentage)
plt.xticks(mfr_delay_percentage.index)
plt.xlabel('Manufacturer', fontsize=20)
plt.ylabel('Carrier Delay Percentage', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
delayed_flight_mfr = delayed_flight.groupby('mfr_name').CARRIER_DELAY.mean().sort_values()
plt.bar(delayed_flight_mfr.index, delayed_flight_mfr.values)
plt.xticks(delayed_flight_mfr.index)
plt.xlabel('Manufacturer', fontsize=20)
plt.ylabel('Average Carrier Delay Minutes', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

## Carrier Delay to aircraft model

In [ ]:
all_flights_model_count = np.array(df.groupby('model_2').DEP_DELAY.count().tolist())
delay_flights_model_count = np.array(df_flight_time.groupby('model_2').delay.count().tolist())
model_names = df.groupby('model_2').DEP_DELAY.count().index.tolist()
model_delay_percentage = pd.DataFrame(delay_flights_model_count / all_flights_model_count, index=model_names, columns=['delay_percentage'])
model_delay_percentage = model_delay_percentage.sort_values(by='delay_percentage')
plt.bar(model_delay_percentage.index, model_delay_percentage.delay_percentage)
plt.xticks(model_delay_percentage.index, rotation=90, fontsize=15)
plt.xlabel('Aircraft Model', fontsize=20)
plt.ylabel('Carrier Delay Percentage', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
delayed_flight_model = delayed_flight.groupby('model_2').CARRIER_DELAY.mean().sort_values()
plt.bar(delayed_flight_model.index, delayed_flight_model.values)
plt.xticks(delayed_flight_model.index, rotation=90)
plt.xlabel('Aircraft Model', fontsize=20)
plt.ylabel('Average Carrier Delay Minutes', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

## Carrier Delay to engine manufacturer

In [ ]:
all_flights_eng_mfr_count = np.array(df.groupby('eng_mfr_name').DEP_DELAY.count().tolist())
delay_flights_eng_mfr_count = np.array(df_flight_time.groupby('eng_mfr_name').delay.count().tolist())
eng_mfr_names = df.groupby('eng_mfr_name').DEP_DELAY.count().index.tolist()
eng_mfr_delay_percentage = pd.DataFrame(delay_flights_eng_mfr_count / all_flights_eng_mfr_count, index=eng_mfr_names, columns=['delay_percentage'])
eng_mfr_delay_percentage = eng_mfr_delay_percentage.sort_values(by='delay_percentage')
plt.bar(eng_mfr_delay_percentage.index, eng_mfr_delay_percentage.delay_percentage)
plt.xticks(eng_mfr_delay_percentage.index, rotation=45)
plt.xlabel('Engine Manufacturer')
plt.ylabel('Carrier Delay Percentage')

In [ ]:
delayed_flight_eng_mfr = delayed_flight.groupby('eng_mfr_name').CARRIER_DELAY.mean().sort_values()
plt.bar(delayed_flight_eng_mfr.index, delayed_flight_eng_mfr.values)
plt.xticks(delayed_flight_eng_mfr.index, rotation=45)
plt.xlabel('Engine Manufacturer')
plt.ylabel('Average Carrier Delay Minutes')

## Carrier Delay to engine model

In [ ]:
all_flights_eng_model_count = np.array(df.groupby('eng_model').DEP_DELAY.count().tolist())
delay_flights_eng_model_count = np.array(df_flight_time.groupby('eng_model').delay.count().tolist())
eng_model_names = df.groupby('eng_model').DEP_DELAY.count().index.tolist()
eng_model_delay_percentage = pd.DataFrame(delay_flights_eng_model_count / all_flights_eng_model_count, index=eng_model_names, columns=['delay_percentage'])
eng_model_delay_percentage = eng_model_delay_percentage.sort_values(by='delay_percentage')
plt.bar(eng_model_delay_percentage.index, eng_model_delay_percentage.delay_percentage)
plt.xticks(eng_model_delay_percentage.index, rotation=90, fontsize=15)
plt.xlabel('Engine Model', fontsize=20)
plt.ylabel('Carrier Delay Percentage', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
delayed_flight_eng_model = delayed_flight.groupby('eng_model').CARRIER_DELAY.mean().sort_values()
plt.bar(delayed_flight_eng_model.index, delayed_flight_eng_model.values)
plt.xticks(delayed_flight_eng_model.index, rotation=90, fontsize=15)
plt.xlabel('Engine Model', fontsize=20)
plt.ylabel('Average Carrier Delay Minutes', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

# Delayed flights on US map

# Calculate delay percentage for airports

In [ ]:
from mpl_toolkits.basemap import Basemap as Basemap

In [ ]:
stations_df = pd.read_csv('./airport_location.csv')

In [ ]:
delay_percentage = pd.read_csv('airport_delay.csv')
delay_percentage['delay_percentage'] = delay_percentage['delay_count'] / delay_percentage['total_count']

In [ ]:
delay_percentage['delay_expect'] = delay_percentage['delay_percentage'] * delay_percentage['avr_delay']

In [ ]:
delay_percentage.sort_values(by='avr_delay', ascending=False)

# Delay by origin airport

In [ ]:
# For delayed flights
delayed_flight = df[df['DEP_DELAY'] > 15].reset_index(drop=True)
# delayed_flight = df[(df['CARRIER_DELAY'] > 0) & (df['DEP_DELAY'] > 15)].reset_index(drop=True)
df_flight_time = pd.DataFrame()
df_flight_time['delay'] = delayed_flight['DEP_DELAY']
df_flight_time['origin'] = delayed_flight['ORIGIN']

In [ ]:
len(df['ORIGIN'].unique().tolist())

In [ ]:
delay_flights_origin_count = pd.DataFrame(df_flight_time.groupby('origin').delay.count()).reset_index().rename(columns={'delay': 'delay_count'})
all_flights_origin_count = pd.DataFrame(df.groupby('ORIGIN').DEP_DELAY.count()).reset_index().rename(columns={'DEP_DELAY': 'total_count', 'ORIGIN': 'origin'})

In [ ]:
delay_percentage = all_flights_origin_count.merge(delay_flights_origin_count, on='origin', how='left')
delay_percentage = delay_percentage.fillna(0)
delay_percentage = delay_percentage.merge(stations_df, left_on='origin', right_on='IATA', how='left').drop(columns=['IATA'])
delay_percentage['delay_percentage'] = delay_percentage['delay_count'] / delay_percentage['total_count']

In [ ]:
delay_percentage = delay_percentage[delay_percentage['total_count'] >= 10]

In [ ]:
# continental US
# reference: https://stackoverflow.com/questions/39742305/how-to-use-basemap-python-to-plot-us-with-50-states
lat = delay_percentage['LATITUDE'].values
lon = delay_percentage['LONGITUDE'].values
size = delay_percentage['delay_percentage'].values
colors = 1 - size
m = Basemap(llcrnrlon=-127,llcrnrlat=22,urcrnrlon=-64,urcrnrlat=51,
        projection='lcc',lat_1=33,lat_2=45,lon_0=-95, resolution='l', epsg=4687)
# m = Basemap(width=12000000,height=9000000,
#             rsphere=(6378137.00,6356752.3142),\
#             resolution='l',area_thresh=1000.,projection='lcc',\
#             lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)
# m.shadedrelief()
# m.bluemarble()
m.arcgisimage(service="ESRI_StreetMap_World_2D", xpixels=1000)
m.drawcoastlines(color='gray')
m.drawcountries(color='black')
m.drawstates(color='grey')
m.scatter(lon, lat, latlon=True,
      c=colors, cmap=plt.cm.hot, alpha=0.8, s=200)
# ax_1 = plt.gca()

# plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
fig = plt.gcf()
fig.set_size_inches(18.5, 12)
# newax = fig.add_axes([0.1, 0.2, 0.2, 0.2], anchor='SW', zorder=10)
# newax.imshow(im)
# newax.axis('off')
plt.show()

In [ ]:
from math import log10

In [ ]:
# continental US
# reference: https://stackoverflow.com/questions/39742305/how-to-use-basemap-python-to-plot-us-with-50-states
lat = delay_percentage['LATITUDE'].values
lon = delay_percentage['LONGITUDE'].values
size = delay_percentage['passenger_count'].apply(lambda x: log10(x)).values
colors = delay_percentage['delay_percentage'].values
m = Basemap(llcrnrlon=-128,llcrnrlat=22,urcrnrlon=-64,urcrnrlat=51,
        projection='lcc',lat_1=33,lat_2=45,lon_0=-96, resolution='l', epsg=4687)
# m.shadedrelief()
# m.bluemarble()
m.arcgisimage(service="ESRI_StreetMap_World_2D", xpixels=950)
m.drawcoastlines(color='gray')
m.drawcountries(color='black')
m.drawstates(color='grey')
m.scatter(lon, lat, latlon=True,
      c=colors, cmap=plt.cm.hot_r, alpha=0.7, s=2.5 ** size)

plt.colorbar(label=r'Delay Percentage')
plt.clim(0, 0.4)

for a in [50000, 500000, 5000000]:
    plt.scatter([], [], c='k', alpha=0.5, s=2.5 ** log10(a),
                label=str(2*a))
plt.legend(scatterpoints=1, frameon=False,
           labelspacing=1, loc='lower right', title='Annual Passenger Count')


fig = plt.gcf()
fig.set_size_inches(18.5, 7.5)

plt.show()

In [ ]:
from PIL import Image

In [ ]:
def fig2data(fig):
    """
    @brief Convert a Matplotlib figure to a 4D numpy array with RGBA channels and return it
    @param fig a matplotlib figure
    @return a numpy 3D array of RGBA values
    """
    # draw the renderer
    fig.canvas.draw()

    # Get the RGBA buffer from the figure
    w,h = fig.canvas.get_width_height()
    buf = np.frombuffer(fig.canvas.tostring_argb(), dtype=np.uint8)
    buf.shape = (w, h, 4)
    # canvas.tostring_argb give pixmap in ARGB mode. Roll the ALPHA channel to have it in RGBA mode
    buf = np.roll (buf, 3, axis=2)
    return buf

In [ ]:
def fig2img(fig):
    """
    @brief Convert a Matplotlib figure to a PIL Image in RGBA format and return it
    @param fig a matplotlib figure
    @return a Python Imaging Library ( PIL ) image
    """
    # put the figure pixmap into a numpy array
    buf = fig2data(fig)
    w, h, d = buf.shape
    return Image.frombytes("RGBA", (w ,h), buf.tostring())

In [ ]:
# Alaska
# reference: https://stackoverflow.com/questions/39742305/how-to-use-basemap-python-to-plot-us-with-50-states
# a = plt.axes([.7, .1, .2, .2])
lat = delay_percentage['LATITUDE'].values
lon = delay_percentage['LONGITUDE'].values
size = delay_percentage['passenger_count'].apply(lambda x: log10(x)).values
colors = delay_percentage['delay_percentage'].values
# plt.figure(edgecolor='r', linewidth=2)
m = Basemap(llcrnrlon=-170,llcrnrlat=50,urcrnrlon=-110,urcrnrlat=72,
        projection='lcc',lat_1=59,lat_2=66,lon_0=-142, resolution='l', epsg=2964)
# m = Basemap(width=12000000,height=9000000,
#             rsphere=(6378137.00,6356752.3142),\
#             resolution='l',area_thresh=1000.,projection='lcc',\
#             lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)
# m.shadedrelief()
# m.bluemarble()
m.arcgisimage(service="ESRI_StreetMap_World_2D", xpixels=950)
# m.drawcoastlines(color='gray')
m.drawcountries(color='black')
m.drawstates(color='grey')
m.scatter(lon, lat, latlon=True,
      c=colors, cmap=plt.cm.hot_r, alpha=0.7, s=2.5 ** size * 4)
ax_2 = plt.gca()
ax_2.text(.7,.15,'Alaska',
        horizontalalignment='center',
        transform=ax_2.transAxes, color='red', fontsize=25)
# plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
fig_2 = plt.gcf()


In [ ]:
# Hawaii
# df_hawaii = delay_percentage[delay_percentage['STATE'] == 'Hawaii']
# reference: https://stackoverflow.com/questions/39742305/how-to-use-basemap-python-to-plot-us-with-50-states
lat = delay_percentage['LATITUDE'].values
lon = delay_percentage['LONGITUDE'].values
size = delay_percentage['passenger_count'].apply(lambda x: log10(x)).values
colors = delay_percentage['delay_percentage'].values
m = Basemap(llcrnrlon=-160,llcrnrlat=18,urcrnrlon=-154,urcrnrlat=23,
        projection='lcc',lat_1=19.5,lat_2=22,lon_0=-156, resolution='l', epsg=2782)
# m = Basemap(width=12000000,height=9000000,
#             rsphere=(6378137.00,6356752.3142),\
#             resolution='l',area_thresh=1000.,projection='lcc',\
#             lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)
# m.shadedrelief()
# m.bluemarble()
m.arcgisimage(service="ESRI_StreetMap_World_2D", xpixels=500)
# m.drawcoastlines(color='gray')
m.drawcountries(color='black')
m.drawstates(color='grey')
m.scatter(lon, lat, latlon=True,
      c=colors, cmap=plt.cm.hot_r, alpha=0.7, s=2.5 ** size * 4)
ax_3 = plt.gca()
ax_3.text(.35,.4,'Hawaii',
        horizontalalignment='center',
        transform=ax_3.transAxes, color='red', fontsize=25)
fig_3 = plt.gcf()

In [ ]:
# Puerto Rico & US Virgin Islands
# df_pr = delay_percentage[(delay_percentage['STATE'] == 'Puerto Rico') | (delay_percentage['STATE'] == 'United States Virgin Islands')]
# reference: https://stackoverflow.com/questions/39742305/how-to-use-basemap-python-to-plot-us-with-50-states
lat = delay_percentage['LATITUDE'].values
lon = delay_percentage['LONGITUDE'].values
size = delay_percentage['passenger_count'].apply(lambda x: log10(x)).values
colors = delay_percentage['delay_percentage'].values
m = Basemap(llcrnrlon=-68,llcrnrlat=17.5,urcrnrlon=-64,urcrnrlat=18.6,
        projection='lcc',lat_1=18,lat_2=18.5,lon_0=-66, resolution='l', epsg=2866)
# m = Basemap(width=12000000,height=9000000,
#             rsphere=(6378137.00,6356752.3142),\
#             resolution='l',area_thresh=1000.,projection='lcc',\
#             lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)
# m.shadedrelief()
# m.bluemarble()
m.arcgisimage(service="ESRI_StreetMap_World_2D", xpixels=1000)
# m.drawcoastlines(color='gray')
m.drawcountries(color='black')
m.drawstates(color='grey')
m.scatter(lon, lat, latlon=True,
      c=colors, cmap=plt.cm.hot_r, alpha=0.7, s=2.5 ** size * 4)
ax_4 = plt.gca()
ax_4.text(.37,.15,'Puerto Rico & US Virgin Islands',
        horizontalalignment='center',
        transform=ax_4.transAxes, color='red', fontsize=17)
fig_4 = plt.gcf()

In [ ]:
# American Samoa
# df_as = delay_percentage[delay_percentage['STATE'] == 'American Samoa']
# reference: https://stackoverflow.com/questions/39742305/how-to-use-basemap-python-to-plot-us-with-50-states
lat = delay_percentage['LATITUDE'].values
lon = delay_percentage['LONGITUDE'].values
size = delay_percentage['passenger_count'].apply(lambda x: log10(x)).values
colors = delay_percentage['delay_percentage'].values
m = Basemap(llcrnrlon=-171,llcrnrlat=-14.4,urcrnrlon=-170.5,urcrnrlat=-14.2,
        projection='lcc',lat_1=-14.4,lat_2=-14.2,lon_0=-170.75, resolution='l', epsg=3102)
# m = Basemap(width=12000000,height=9000000,
#             rsphere=(6378137.00,6356752.3142),\
#             resolution='l',area_thresh=1000.,projection='lcc',\
#             lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)
# m.shadedrelief()
# m.bluemarble()
m.arcgisimage(service="ESRI_StreetMap_World_2D", xpixels=1000)
# m.drawcoastlines(color='gray')
m.drawcountries(color='black')
m.drawstates(color='grey')
m.scatter(lon, lat, latlon=True,
      c=colors, cmap=plt.cm.hot_r, alpha=0.7, s=2.5 ** size * 100 / 16)
ax_5 = plt.gca()
ax_5.text(.35,.75,'American Samoa',
        horizontalalignment='center',
        transform=ax_5.transAxes, color='red', fontsize=25)
fig_5 = plt.gcf()

In [ ]:
# Guam
# reference: https://stackoverflow.com/questions/39742305/how-to-use-basemap-python-to-plot-us-with-50-states
lat = delay_percentage['LATITUDE'].values
lon = delay_percentage['LONGITUDE'].values
size = delay_percentage['passenger_count'].apply(lambda x: log10(x)).values
colors = delay_percentage['delay_percentage'].values
m = Basemap(llcrnrlon=144.5,llcrnrlat=13.2,urcrnrlon=145,urcrnrlat=13.7,
        projection='lcc',lat_1=13.3,lat_2=13.6,lon_0=144.75, resolution='l', epsg=4675)
# m = Basemap(width=12000000,height=9000000,
#             rsphere=(6378137.00,6356752.3142),\
#             resolution='l',area_thresh=1000.,projection='lcc',\
#             lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)
# m.shadedrelief()
# m.bluemarble()
m.arcgisimage(service="ESRI_StreetMap_World_2D", xpixels=1000)
# m.drawcoastlines(color='gray')
m.drawcountries(color='black')
m.drawstates(color='grey')
m.scatter(lon, lat, latlon=True,
      c=colors, cmap=plt.cm.hot_r, alpha=0.7, s=2.5 ** size * 100 / 16)
ax_6 = plt.gca()
ax_6.text(.35,.75,'Guam',
        horizontalalignment='center',
        transform=ax_6.transAxes, color='red', fontsize=25)
fig_6 = plt.gcf()

# Airport Average Delay Minutes

In [ ]:
passenger = pd.read_csv('airport_passenger.csv')
passenger.iloc[:, 1] = passenger.iloc[:, 1].str.replace(',','').apply(lambda x: int(x))

In [ ]:
passenger = passenger.rename(columns={'17_Enplanements': 'passenger_count'})

In [ ]:
avr_delay = avr_delay.merge(passenger, left_on='origin', right_on='IATA', how='left').drop(columns=['IATA'])

In [ ]:
all_flights_origin_count = pd.DataFrame(df.groupby('ORIGIN').DEP_DELAY.count()).reset_index().rename(columns={'DEP_DELAY': 'total_count', 'ORIGIN': 'origin'})
delay_flights_origin_count = pd.DataFrame(df_flight_time.groupby('origin').delay.count()).reset_index().rename(columns={'delay': 'delay_count'})

all_flights_average_delay = pd.DataFrame(df.groupby('ORIGIN').DEP_DELAY.mean().sort_values()).reset_index().rename(columns={'DEP_DELAY': 'all_avr_delay', 'ORIGIN': 'origin'})
delay_flights_average_delay = pd.DataFrame(df_flight_time.groupby('origin').delay.mean()).reset_index().rename(columns={'delay': 'avr_delay'})

delay_percentage = all_flights_origin_count.merge(delay_flights_origin_count, on='origin', how='left')
delay_percentage = delay_percentage.fillna(0)

avr_delay = all_flights_average_delay.merge(delay_flights_average_delay, on='origin', how='left')
avr_delay = avr_delay.fillna(0)

avr_delay = avr_delay.merge(delay_percentage, on='origin', how='left')
avr_delay = avr_delay.merge(stations_df, left_on='origin', right_on='IATA', how='left').drop(columns=['IATA'])

In [ ]:
avr_delay = avr_delay[avr_delay['total_count'] >= 10]

In [ ]:
# set the stations 3 std of avr_day far away mean to mean + 3 std
inx = avr_delay[np.abs(avr_delay.avr_delay - avr_delay.avr_delay.mean()) > (3 * avr_delay.avr_delay.std())].index.tolist()
set_value = avr_delay.avr_delay.mean() + 3 * avr_delay.avr_delay.std()
for i in inx:
    avr_delay.loc[i, 'avr_delay'] = set_value

In [ ]:
avr_delay.sort_values(by='avr_delay', ascending=False)

# Average Delay Minutes for All Flights

In [ ]:
# continental US
# reference: https://stackoverflow.com/questions/39742305/how-to-use-basemap-python-to-plot-us-with-50-states
lat = avr_delay['LATITUDE'].values
lon = avr_delay['LONGITUDE'].values
colors = avr_delay['all_avr_delay'].values
m = Basemap(llcrnrlon=-128,llcrnrlat=22,urcrnrlon=-64,urcrnrlat=51,
        projection='lcc',lat_1=33,lat_2=45,lon_0=-96, resolution='l', epsg=4687)
# m.shadedrelief()
# m.bluemarble()
m.arcgisimage(service="ESRI_StreetMap_World_2D", xpixels=950)
m.drawcoastlines(color='gray')
m.drawcountries(color='black')
m.drawstates(color='grey')
m.scatter(lon, lat, latlon=True,
      c=colors, cmap=plt.cm.hot_r, alpha=0.8, s=200)

plt.colorbar(label=r'Average Delay Minites of All Flights')
plt.clim(avr_delay['all_avr_delay'].min(), avr_delay['all_avr_delay'].max())

fig = plt.gcf()
fig.set_size_inches(18.5, 7.5)

plt.show()

In [ ]:
from math import log10

# Average Delay Minutes for Delayed Flights

In [ ]:
avr_delay['delay_count'].values / avr_delay['total_count'].values * 40

In [ ]:
# continental US
# reference: https://stackoverflow.com/questions/39742305/how-to-use-basemap-python-to-plot-us-with-50-states
lat = avr_delay['LATITUDE'].values
lon = avr_delay['LONGITUDE'].values
# size = avr_delay['avr_delay'].apply(lambda x: log10(x)).values
size = avr_delay['passenger_count'].apply(lambda x: log10(x)).values
# size = avr_delay['delay_count'].values / avr_delay['total_count'].values
colors = avr_delay['avr_delay'].values
m = Basemap(llcrnrlon=-128,llcrnrlat=22,urcrnrlon=-64,urcrnrlat=51,
        projection='lcc',lat_1=33,lat_2=45,lon_0=-96, resolution='l', epsg=4687)
# m.shadedrelief()
# m.bluemarble()
m.arcgisimage(service="ESRI_StreetMap_World_2D", xpixels=950)
m.drawcoastlines(color='gray')
m.drawcountries(color='black')
m.drawstates(color='grey')
m.scatter(lon, lat, latlon=True,
      c=colors, cmap=plt.cm.hot_r, alpha=0.8, s= 2.5 ** size)

plt.colorbar(label=r'Average Delay Minites of Delayed Flights')
# plt.clim(log10(avr_delay['avr_delay'].min()), log10(avr_delay['avr_delay'].max()))
plt.clim(avr_delay['avr_delay'].min(), avr_delay['avr_delay'].max())

# make legend with dummy points
for a in [50000, 500000, 5000000]:
    plt.scatter([], [], c='k', alpha=0.5, s=2.5 ** log10(a),
                label=str(2*a))
plt.legend(scatterpoints=1, frameon=False,
           labelspacing=1, loc='lower right', title='Annual Passenger Count');


fig = plt.gcf()
fig.set_size_inches(18.5, 7.5)

plt.show()

In [ ]:
# Alaska
# reference: https://stackoverflow.com/questions/39742305/how-to-use-basemap-python-to-plot-us-with-50-states
# a = plt.axes([.7, .1, .2, .2])
lat = avr_delay['LATITUDE'].values
lon = avr_delay['LONGITUDE'].values
size = avr_delay['passenger_count'].apply(lambda x: log10(x)).values
colors = avr_delay['avr_delay'].values
# plt.figure(edgecolor='r', linewidth=2)
m = Basemap(llcrnrlon=-170,llcrnrlat=50,urcrnrlon=-110,urcrnrlat=72,
        projection='lcc',lat_1=59,lat_2=66,lon_0=-142, resolution='l', epsg=2964)
# m = Basemap(width=12000000,height=9000000,
#             rsphere=(6378137.00,6356752.3142),\
#             resolution='l',area_thresh=1000.,projection='lcc',\
#             lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)
# m.shadedrelief()
# m.bluemarble()
m.arcgisimage(service="ESRI_StreetMap_World_2D", xpixels=950)
# m.drawcoastlines(color='gray')
m.drawcountries(color='black')
m.drawstates(color='grey')
m.scatter(lon, lat, latlon=True,
      c=colors, cmap=plt.cm.hot_r, alpha=0.8, s= 2.5 ** size * 4)
ax_2 = plt.gca()
ax_2.text(.7,.15,'Alaska',
        horizontalalignment='center',
        transform=ax_2.transAxes, color='red', fontsize=25)
# plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
fig_2 = plt.gcf()


In [ ]:
# Hawaii
# df_hawaii = delay_percentage[delay_percentage['STATE'] == 'Hawaii']
# reference: https://stackoverflow.com/questions/39742305/how-to-use-basemap-python-to-plot-us-with-50-states
lat = avr_delay['LATITUDE'].values
lon = avr_delay['LONGITUDE'].values
size = avr_delay['passenger_count'].apply(lambda x: log10(x)).values
colors = avr_delay['avr_delay'].values
m = Basemap(llcrnrlon=-160,llcrnrlat=18,urcrnrlon=-154,urcrnrlat=23,
        projection='lcc',lat_1=19.5,lat_2=22,lon_0=-156, resolution='l', epsg=2782)
# m = Basemap(width=12000000,height=9000000,
#             rsphere=(6378137.00,6356752.3142),\
#             resolution='l',area_thresh=1000.,projection='lcc',\
#             lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)
# m.shadedrelief()
# m.bluemarble()
m.arcgisimage(service="ESRI_StreetMap_World_2D", xpixels=500)
# m.drawcoastlines(color='gray')
m.drawcountries(color='black')
m.drawstates(color='grey')
m.scatter(lon, lat, latlon=True,
      c=colors, cmap=plt.cm.hot_r, alpha=0.8, s= 2.5 ** size * 4)
ax_3 = plt.gca()
ax_3.text(.35,.4,'Hawaii',
        horizontalalignment='center',
        transform=ax_3.transAxes, color='red', fontsize=25)
fig_3 = plt.gcf()

In [ ]:
# Puerto Rico & US Virgin Islands
# df_pr = delay_percentage[(delay_percentage['STATE'] == 'Puerto Rico') | (delay_percentage['STATE'] == 'United States Virgin Islands')]
# reference: https://stackoverflow.com/questions/39742305/how-to-use-basemap-python-to-plot-us-with-50-states
lat = avr_delay['LATITUDE'].values
lon = avr_delay['LONGITUDE'].values
colors = avr_delay['avr_delay'].values
size = avr_delay['passenger_count'].apply(lambda x: log10(x)).values
m = Basemap(llcrnrlon=-68,llcrnrlat=17.5,urcrnrlon=-64,urcrnrlat=18.6,
        projection='lcc',lat_1=18,lat_2=18.5,lon_0=-66, resolution='l', epsg=2866)
# m = Basemap(width=12000000,height=9000000,
#             rsphere=(6378137.00,6356752.3142),\
#             resolution='l',area_thresh=1000.,projection='lcc',\
#             lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)
# m.shadedrelief()
# m.bluemarble()
m.arcgisimage(service="ESRI_StreetMap_World_2D", xpixels=1000)
# m.drawcoastlines(color='gray')
m.drawcountries(color='black')
m.drawstates(color='grey')
m.scatter(lon, lat, latlon=True,
      c=colors, cmap=plt.cm.hot_r, alpha=0.8, s=2.5 ** size * 4)
ax_4 = plt.gca()
ax_4.text(.37,.15,'Puerto Rico & US Virgin Islands',
        horizontalalignment='center',
        transform=ax_4.transAxes, color='red', fontsize=17)
fig_4 = plt.gcf()

In [ ]:
# American Samoa
# df_as = delay_percentage[delay_percentage['STATE'] == 'American Samoa']
# reference: https://stackoverflow.com/questions/39742305/how-to-use-basemap-python-to-plot-us-with-50-states
lat = avr_delay['LATITUDE'].values
lon = avr_delay['LONGITUDE'].values
colors = avr_delay['avr_delay'].values
size = avr_delay['passenger_count'].apply(lambda x: log10(x)).values
m = Basemap(llcrnrlon=-171,llcrnrlat=-14.4,urcrnrlon=-170.5,urcrnrlat=-14.2,
        projection='lcc',lat_1=-14.4,lat_2=-14.2,lon_0=-170.75, resolution='l', epsg=3102)
# m = Basemap(width=12000000,height=9000000,
#             rsphere=(6378137.00,6356752.3142),\
#             resolution='l',area_thresh=1000.,projection='lcc',\
#             lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)
# m.shadedrelief()
# m.bluemarble()
m.arcgisimage(service="ESRI_StreetMap_World_2D", xpixels=1000)
# m.drawcoastlines(color='gray')
m.drawcountries(color='black')
m.drawstates(color='grey')
m.scatter(lon, lat, latlon=True,
      c=colors, cmap=plt.cm.hot_r, alpha=0.8, s=2.5 ** size * 100 / 16)
ax_5 = plt.gca()
ax_5.text(.35,.75,'American Samoa',
        horizontalalignment='center',
        transform=ax_5.transAxes, color='red', fontsize=25)
fig_5 = plt.gcf()

In [ ]:
# Guam
# reference: https://stackoverflow.com/questions/39742305/how-to-use-basemap-python-to-plot-us-with-50-states
lat = avr_delay['LATITUDE'].values
lon = avr_delay['LONGITUDE'].values
colors = avr_delay['avr_delay'].values
size = avr_delay['passenger_count'].apply(lambda x: log10(x)).values
m = Basemap(llcrnrlon=144.5,llcrnrlat=13.2,urcrnrlon=145,urcrnrlat=13.7,
        projection='lcc',lat_1=13.3,lat_2=13.6,lon_0=144.75, resolution='l', epsg=4675)
# m = Basemap(width=12000000,height=9000000,
#             rsphere=(6378137.00,6356752.3142),\
#             resolution='l',area_thresh=1000.,projection='lcc',\
#             lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)
# m.shadedrelief()
# m.bluemarble()
m.arcgisimage(service="ESRI_StreetMap_World_2D", xpixels=1000)
# m.drawcoastlines(color='gray')
m.drawcountries(color='black')
m.drawstates(color='grey')
m.scatter(lon, lat, latlon=True,
      c=colors, cmap=plt.cm.hot_r, alpha=0.8, s=2.5 ** size * 100 / 16)
ax_6 = plt.gca()
ax_6.text(.35,.75,'Guam',
        horizontalalignment='center',
        transform=ax_6.transAxes, color='red', fontsize=25)
fig_6 = plt.gcf()

In [ ]:
pd.Series(statenames).unique()

In [ ]:
delay_percentage.STATE.unique()

In [ ]:
list(map(lambda x: (x[0] + 2, x[1]-2), a))
# lambda kv: (-kv[1], kv[0])

In [ ]:
enumerate(m.states)

In [ ]:
statenames

In [ ]:
stations_df.head()

In [ ]:
delayed_flight_carrier = delayed_flight.groupby('OP_CARRIER').DEP_DELAY.mean().sort_values()
plt.bar(delayed_flight_carrier.index, delayed_flight_carrier.values)
plt.xticks(delayed_flight_carrier.index, fontsize=15)
plt.xlabel('Carrier', fontsize=20)
plt.ylabel('Average Delay Minutes', fontsize=20)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
df[(df['TAIL_NUM'] != 'PLANET') & (df['TAIL_NUM'] != 'N101NK') & (df['TAIL_NUM'] != 'N999FR') & (df['TAIL_NUM'] != 'N700TA') & (df['TAIL_NUM'] != 'N187SW') & (df['TAIL_NUM'] != 'N399FR')].shape

In [ ]:
df['ARR_DELAY_GROUP'].head()

In [ ]:
% matplotlib inline
# plt.scatter(df['ARR_DELAY_GROUP'], df['DISTANCE'], s=0.1, alpha=0.3)
data = pd.concat([df['DEP_DELAY_GROUP'], df['DISTANCE_GROUP']], axis=1)
sns.jointplot(x='DEP_DELAY_GROUP', y='DISTANCE_GROUP', data=data.loc[:1000], kind="kde")

In [ ]:
% matplotlib inline
# plt.scatter(df['ARR_DELAY_GROUP'], df['DISTANCE'], s=0.1, alpha=0.3)
data = pd.concat([df['DEP_DELAY_GROUP'], df['DEP_TIME_BLK'].apply(lambda x: int(x[:2]))], axis=1)
sns.jointplot(x='DEP_DELAY_GROUP', y='DEP_TIME_BLK', data=data.loc[:10000], kind="kde")

In [ ]:
df['DEP_TIME_BLK']

In [ ]:
data

In [ ]:
uniform_data = np.random.rand(10, 12)

In [ ]:
uniform_data